# Test notebook

__A. J. Tropiano [tropiano.4@osu.edu]__<br/>
__March 17, 2022__

The purpose of this notebook is for testing out code and/or scripts.

_Last update: April 8, 2022_

In [1]:
# Python imports
from matplotlib.offsetbox import AnchoredText
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import time

In [2]:
# Imports from A.T. codes
import densities
from momentum_distributions import MomentumDistribution
from potentials import Potential
# You could import modules but I want a list of scripts here
# Check these off the list as you go through Jupyter notebooks generating
# figures
# import dmd, snmd, pmd
import modules.a2
import modules.figures
import modules.fourier_transform
import modules.integration
import modules.labels
import modules.long_distance_operators
from magnus import Magnus, MagnusSplit
import modules.momentum_projection_operator
import modules.phase_shifts
import modules.sdt
from srg import SRG, get_transformation
import modules.tools
import modules.wave_function
# add missing ones

For now let's illustrate the overhaul by testing scripts and showing how to use them.

### Test `vnn.py`

In [3]:
kvnn = 900
channel = '3S1'
kmax, kmid, ntot = 30.0, 4.0, 120

In [4]:
# __init__ (check)

potential = Potential(kvnn, channel, kmax, kmid, ntot)
print(potential.kvnn, potential.coupled_channel_bool, 
      potential.potential_directory)

900 True ../potentials/vsrg_macos/vsrg_kvnn_900_lam12.0_kmax30_kmid4_ntot120/


In [5]:
# load_mesh (check)

k_array, k_weights = potential.load_mesh()

In [6]:
# load_potential (check)

V_matrix = potential.load_potential()

In [7]:
# load_hamiltonian (check)

H_matrix = potential.load_hamiltonian()

### Test `srg.py`

In [8]:
# run_srg for Wegner (check)

generator = 'Wegner'
srg = SRG(kvnn, channel, kmax, kmid, ntot, generator)
lambda_array = np.array([3.0])
d = srg.srg_evolve(lambda_array, save=True)

_____________________________________________________________________________________
Done evolving to final \lambda = 3 fm^-1 after 0.0035 minutes.
_____________________________________________________________________________________

Specifications:

kvnn = 900, channel = 3S1
kmax = 30.0, kmid = 4.0, ntot = 120
method = SRG, generator = Wegner


In [9]:
# run_srg for T (check)

generator = 'T'
srg = SRG(kvnn, channel, kmax, kmid, ntot, generator)
d = srg.srg_evolve(lambda_array, save=True)

_____________________________________________________________________________________
Done evolving to final \lambda = 3 fm^-1 after 0.0033 minutes.
_____________________________________________________________________________________

Specifications:

kvnn = 900, channel = 3S1
kmax = 30.0, kmid = 4.0, ntot = 120
method = SRG, generator = T


In [10]:
# run_srg for Block-diag (check)

generator = 'Block-diag'
lambda_bd_array = np.array([3.0])
lambda_array = np.array([1.0])
srg = SRG(kvnn, channel, kmax, kmid, ntot, generator)
d = srg.srg_evolve(lambda_array, lambda_bd_array, save=True)

_____________________________________________________________________________________
Done evolving to final \lambda = 1 fm^-1 after 0.3994 minutes.
_____________________________________________________________________________________

Specifications:

kvnn = 900, channel = 3S1
kmax = 30.0, kmid = 4.0, ntot = 120
method = SRG, generator = Block-diag
Final \Lambda_BD = 3 fm^-1


In [11]:
# compare evolved Hamiltonian using get_transformation (check)

H_initial = potential.load_hamiltonian()
H_evolved = potential.load_hamiltonian('srg', 'Wegner', 3.0)
U_matrix = get_transformation(H_initial, H_evolved)

### Test `magnus.py`

In [12]:
# Might be smart to test a really soft potential here for speed
kvnn = 111
channel = '1S0'
kmax, kmid, ntot = 10.0, 2.0, 120

In [13]:
# Do default Magnus arguments
k_max = 2
ds = 1e-5

In [14]:
# run_magnus for Wegner (check)

generator = 'Wegner'
lambda_array = np.array([3.0])
magnus = Magnus(kvnn, channel, kmax, kmid, ntot, generator)
d = magnus.magnus_evolve(lambda_array, k_max=k_max, ds=ds)
print(d[3.0])

# run_srg for Wegner
srg = SRG(kvnn, channel, kmax, kmid, ntot, generator)
d = srg.srg_evolve(lambda_array)
print(d[3.0])

_____________________________________________________________________________________
Done evolving to final \lambda = 3 fm^-1 after 0.0444 minutes.
_____________________________________________________________________________________

Specifications:

kvnn = 111, channel = 1S0
kmax = 10.0, kmid = 2.0, ntot = 120
method = Magnus, generator = Wegner
k_max = 2, ds = 1.0e-05
[[ 6.22375968e-07 -1.22803393e-08 -3.77512123e-08 ...  3.02466439e-17
   2.32844608e-17  1.49598933e-17]
 [-1.22803393e-08  1.72026235e-05 -3.03132572e-07 ...  2.42873836e-16
   1.86968933e-16  1.20124513e-16]
 [-3.77512123e-08 -3.03132572e-07  1.03355214e-04 ...  7.46645308e-16
   5.74780087e-16  3.69286283e-16]
 ...
 [ 3.02466474e-17  2.42873849e-16  7.46645255e-16 ...  9.91800753e+01
  -3.67712230e-03 -2.39842920e-03]
 [ 2.32844606e-17  1.86968931e-16  5.74780087e-16 ... -3.67712230e-03
   9.96645753e+01 -1.92725662e-03]
 [ 1.49598943e-17  1.20124513e-16  3.69286286e-16 ... -2.39842920e-03
  -1.92725662e-03  9.9935

In [15]:
# # run_magnus for T (check)

# generator = 'T'
# lambda_array = np.array([6.0, 3.0])

# d = potential.run_magnus(generator, lambda_array, k_max=k_max, ds=ds,
#                          save=False)
# print(d[3.0])

# # run_srg for Wegner
# d = potential.run_srg(generator, lambda_array, save=False)
# print(d[3.0])

In [16]:
# # run_magnus for Block-diag

# generator = 'Block-diag'
# lambda_bd_array = np.array([6.0, 3.0])
# lambda_array = np.array([3.0])

# d = potential.run_magnus(generator, lambda_array, lambda_bd_array, 
#                          k_max=k_max, ds=ds, save=False)
# print(d[3.0][3.0])

# # run_srg for Block-diag (check)
# d = potential.run_srg(generator, lambda_array, lambda_bd_array, save=False)
# print(d[3.0][3.0])

In [18]:
# Test that Magnus_split at least works (check)

magnus_split = MagnusSplit(kvnn, channel, kmax, kmid, ntot, generator)
d = magnus_split.magnus_split_evolve(lambda_array, k_max=k_max, ds=ds)
print(d[3.0])

[[ 6.22375968e-07 -1.22803393e-08 -3.77512123e-08 ...  9.74524521e-30
   5.74103218e-30  3.17673282e-30]
 [-1.22803393e-08  1.72026235e-05 -3.03132573e-07 ...  7.82545284e-29
   4.61006119e-29  2.55092330e-29]
 [-3.77512123e-08 -3.03132573e-07  1.03355214e-04 ...  2.40608978e-28
   1.41745424e-28  7.84331691e-29]
 ...
 [ 9.74524521e-30  7.82545284e-29  2.40608978e-28 ...  9.91800753e+01
  -3.67712230e-03 -2.39842920e-03]
 [ 5.74103218e-30  4.61006119e-29  1.41745424e-28 ... -3.67712230e-03
   9.96645753e+01 -1.92725662e-03]
 [ 3.17673282e-30  2.55092330e-29  7.84331691e-29 ... -2.39842920e-03
  -1.92725662e-03  9.99355553e+01]]


### Testing momentum distributions code

In [ ]:
# # kvnn = 6
# # kmax, kmid, ntot = 15.0, 3.0, 120
# kvnn = 111
# kmax, kmid, ntot = 10.0, 2.0, 120
# channels = ('1S0', '3S1')
# generator = 'Wegner'
# lamb = 1.5

In [ ]:
# # This works comparing to old snmd.py and pmd.py
# md = MomentumDistributions(kvnn, kmax, kmid, ntot)
# md.save_deltaU_funcs(channels, generator, lamb)

In [ ]:
# chdir(old_code_dir)
# snmd = single_nucleon_momentum_distributions(kvnn, channels, lamb, kmax, kmid,
#                                              ntot)
# chdir(cwd)

In [ ]:
# md.save_deuteron_deltaU_funcs(generator, lamb)

In [ ]:
# chdir(old_code_dir)
# dmd = deuteron_momentum_distributions(kvnn, lamb, kmax, kmid, ntot)
# chdir(cwd)

In [ ]:
# Test figure_graphics.py

In [ ]:
# Test figure_labels.py

In [ ]:
# Test fourier_transform.py

In [ ]:
# Test integration.py

In [ ]:
# Test long_distance_operators.py

In [ ]:
# Test momentum_projection_operator.py

In [ ]:
# Test phase_shifts.py

In [ ]:
# Test sdt.py

In [ ]:
# Test srg.py

In [ ]:
# Test tools.py

In [ ]:
# Test wave_function.py